In [6]:
from spinbox import *
from itertools import count
seeder = count(0,1) # generates a list of counting numbers for rng "seeds" (not really seeds in Numpy, but indices of seeds)

def npr(name, thing):
    print(name,"\n",thing,"\n"+16*"-")

# Tutorial 1 : Simple calculations in the full Hilbert space

For GFMC-style calculations one should use `HilbertState` and `HilbertOperator` classes, named as such because the entire basis of the Hilbert space is available. That is, a state is a linear combination of tensor product states.

To start, let's instantiate some random GFMC-style states for a 2-particle system with just spin (no isospin) and compute stuff with them.

In [7]:
ket_1 = HilbertState(n_particles=2, ketwise=True, isospin=False).random(seed=next(seeder))
print(ket_1)

HilbertState ket of 2 particles:
[[ 0.06750061-0.2875841j ]
 [-0.07092296+0.19412905j]
 [ 0.34382285+0.70007675j]
 [ 0.05631758+0.50845808j]]


We have instatiated a random complex-values vector with 4 components corresponding to the states $|\uparrow\uparrow\rangle, |\uparrow\downarrow\rangle,|\downarrow\uparrow\rangle,|\downarrow\downarrow\rangle$.

Note that `ketwise=True` makes a ket state. Setting this to false makes a bra state. Applying the `.dagger()` method returns the Hermitian conjugate and changes this attribute. 

We can instatiate another state and do some typical calculations.

In [8]:
bra_2 = HilbertState(n_particles=2, ketwise=False, isospin=False).random(seed=next(seeder))
print(bra_2)

HilbertState bra of 2 particles:
[[ 0.16769553+0.43932604j  0.39869211+0.2166043j   0.16034536-0.26055781j -0.63236008+0.28198891j]]


We can add and subtract these states.

In [4]:
npr("| state 1 > - | state 1 > = " , ket_1 - ket_1)
npr("| state 1 > + | state 2 > = " , ket_1 + bra_2.dagger())

| state 1 > - | state 1 > =  
 HilbertState ket of 2 particles:
[[0.+0.j]
 [0.+0.j]
 [0.+0.j]
 [0.+0.j]] 
----------------
| state 1 > + | state 2 > =  
 HilbertState ket of 2 particles:
[[ 0.23519615-0.72691013j]
 [ 0.32776915-0.02247525j]
 [ 0.50416821+0.96063456j]
 [-0.57604249+0.22646917j]] 
----------------


The `*` operator does most sensible multiplication. States are normalized by default.

You can multiply a state by a scalar number, but the scalar must go on the right, not the left. The reason for this has to do with Python's multiplication methods (I didn't want to define a `__rmul__` method for this).

In [5]:
print(ket_1 * 3.0) # reversing this order will give an error

HilbertState ket of 2 particles:
[[ 0.20250184-0.86275229j]
 [-0.21276888+0.58238715j]
 [ 1.03146854+2.10023025j]
 [ 0.16895275+1.52537425j]]


A bra times a ket returns a scalar. A ket times a bra returns a `HilbertOperator`.

In [6]:
npr("< state 1 | state 1 > = " , ket_1.dagger() * ket_1)
npr("< state 2 | state 1 > = " , bra_2 * ket_1)
npr("| state 1 > < state 2 | = " , ket_1 * bra_2)

< state 1 | state 1 > =  
 [[1.+0.j]] 
----------------
< state 2 | state 1 > =  
 [[0.12588545-0.23951561j]] 
----------------
| state 1 > < state 2 | =  
 HilbertOperator
Re=
[[ 0.13766273  0.08920391 -0.06410887  0.03841083]
 [-0.09717941 -0.07032561  0.03920967 -0.00989339]
 [-0.24990439 -0.01456018  0.23754086 -0.41483372]
 [-0.21393467 -0.08768083  0.14151299 -0.17899253]]
Im:
[[-0.01857179 -0.10003659 -0.06370059  0.20089113]
 [ 0.00139627  0.0620355   0.04960722 -0.14275895]
 [ 0.26845007  0.35358859  0.02266833 -0.34574636]
 [ 0.11000793  0.21491686  0.06685491 -0.30564766]] 
----------------


This can be used to compute the density matrix for a state, but there is also a method `.density()` that does this.

In [7]:
density = ket_1.density()
npr("| state 1 > < state 1 | = " , density)
npr("diag(| state 1 > < state 1 |) = ", np.linalg.eig(density.coefficients))

| state 1 > < state 1 | =  
 HilbertOperator
Re=
[[ 0.08726095 -0.06061577 -0.17812269 -0.14242299]
 [-0.06061577  0.04271615  0.1115203   0.09471228]
 [-0.17812269  0.1115203   0.60832161  0.37532296]
 [-0.14242299  0.09471228  0.37532296  0.26170129]]
Im:
[[ 0.          0.00729249 -0.14613359 -0.05051727]
 [-0.00729249  0.          0.11639752  0.04699423]
 [ 0.14613359 -0.11639752  0.         -0.13539287]
 [ 0.05051727 -0.04699423  0.13539287  0.        ]] 
----------------
diag(| state 1 > < state 1 |) =  
 (array([ 3.66359243e-17-2.78268792e-18j,  1.00000000e+00-3.19117816e-17j,  1.48775219e-17-2.51248168e-17j, -2.57842133e-17+1.04129088e-17j]), array([[ 0.95537378+0.j        , -0.22837713-0.18736283j, -0.05500913-0.11455024j, -0.17415788+0.20346611j],
       [ 0.06344718+0.00763312j,  0.14298396+0.1492372j ,  0.11836996-0.22835441j,  0.82714387+0.j        ],
       [ 0.18644293-0.1529596j ,  0.77994975+0.j        , -0.50530291+0.21866344j,  0.11524683+0.24978252j],
       [ 0.1490

Since the `__mul__` rule is defined for multiple classes, we can actually have an error for improper multiplication. For simplicity, both ket times ket and bra times bra return this error.

In [8]:
try:
    print("| state 1 > | state 2 > = " , ket_1 * bra_2.dagger())
except:
    print("This raises an error.")

This raises an error.


Finally, we can list multiplications together as usual. 

In [9]:
outer_product = ket_1 * bra_2
npr("< state 1 | ( | state 1 > < state 2 | ) | state 2> = " , ket_1.dagger() * outer_product * bra_2.dagger())

< state 1 | ( | state 1 > < state 2 | ) | state 2> =  
 [[1.-4.16333634e-17j]] 
----------------


We can make our own operator from scratch too. Operators are always instantiated to the identity. If you want to instantiate to zero, use the `.zero()` method. We can apply the Pauli spin operators to individual particles, and the isospin operators (if the isospin DOFs are included).

In [10]:
operator = HilbertOperator(n_particles=2, isospin=False).apply_sigma(particle_index=0, dimension=0)
print(operator)

HilbertOperator
Re=
[[0. 0. 1. 0.]
 [0. 0. 0. 1.]
 [1. 0. 0. 0.]
 [0. 1. 0. 0.]]
Im:
[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]


In the tensor product space formalism, this is the one-body operator $\hat{\sigma}_{x} \otimes I := \hat{\sigma}_{1x}$. Note that indices start at zero: particle $i$ is indexed $i-1$.

If we include isospin, we can apply $\tau$ operators too.

In [11]:
operator = HilbertOperator(n_particles=2, isospin=True).apply_sigma(particle_index=0, dimension=0).apply_tau(particle_index=1,dimension=2)
print(operator)

HilbertOperator
Re=
[[ 0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0. -1.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0. -1.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0. -1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0. -1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. -1.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. -1.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0. 

In the tensor product space formalism, this is $\hat{\sigma}_{x} \otimes \hat{\tau}_z := \hat{\sigma}_{1x}\hat{\tau}_{2z}$. 

In the full Hilbert space, states and operators can be added together to produce states and operators respectively. This is not the case with `ProductState` and `ProductOperator`, which I will get to later.

Let's construct the two-body spin interaction operator for our two particles: $\vec{\sigma}_1 \cdot \vec{\sigma}_2$.

In [12]:
si_dot_sj = HilbertOperator(n_particles=2,isospin=False).zero()
for dimension in (0,1,2):
    si_dot_sj += HilbertOperator(n_particles=2,isospin=False).apply_sigma(0,dimension).apply_sigma(1,dimension)

print(si_dot_sj)

HilbertOperator
Re=
[[ 1.  0.  0.  0.]
 [ 0. -1.  2.  0.]
 [ 0.  2. -1.  0.]
 [ 0.  0.  0.  1.]]
Im:
[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]


We can use the built-in `.exp()` method to exponentiate a `HilbertOperator` via Pade approximant. For instance, making an imaginary-time propagator object from $\vec{\sigma}_1 \cdot \vec{\sigma}_2$ might look something like 

In [13]:
delta_tau = 0.1j
propagator = si_dot_sj.scale(-delta_tau).exp()
print(propagator)

HilbertOperator
Re=
[[0.99500417 0.         0.         0.        ]
 [0.         0.97517033 0.01983384 0.        ]
 [0.         0.01983384 0.97517033 0.        ]
 [0.         0.         0.         0.99500417]]
Im:
[[-0.09983342  0.          0.          0.        ]
 [ 0.          0.0978434  -0.19767681  0.        ]
 [ 0.         -0.19767681  0.0978434   0.        ]
 [ 0.          0.          0.         -0.09983342]]


However, `spinbox` has classes for propagators, so you probably want to use those instead!